In [1]:
import pandas as pd
import time
import os
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

print("🔄 Bắt đầu đọc dữ liệu Parquet...")
start_read_time = time.time()  # Đo thời gian đọc
parquet_folder = r"C:\Khue\TDN\data\processed"

try:
    all_files_in_folder = os.listdir(parquet_folder)
    parquet_files = [
        os.path.join(parquet_folder, f)
        for f in all_files_in_folder
        if f.startswith("sanluong_") and f.endswith(".parquet")
    ]

    if not parquet_files:
        print(f"⚠️ Không tìm thấy file Parquet trong: {parquet_folder}")
        df_all = pd.DataFrame()
    else:
        df_list = []
        print(f"🔍 Tìm thấy {len(parquet_files)} file Parquet. Bắt đầu đọc...")
        for i, f in enumerate(parquet_files):
            try:
                df_temp = pd.read_parquet(f, engine="pyarrow")
                # Chỉ đọc các cột thực sự cần thiết ngay từ đầu để tiết kiệm bộ nhớ
                required_cols_read = ["CTDL", "NMTD", "MADIEMDO", "ENDTIME", "CS"]
                if all(col in df_temp.columns for col in required_cols_read):
                    df_list.append(df_temp[required_cols_read])  # Chỉ lấy cột cần thiết
                else:
                    print(f"   ⚠️ File {os.path.basename(f)} thiếu cột, bỏ qua.")
            except Exception as e:
                print(f"❌ Lỗi đọc file {os.path.basename(f)}: {e}")

        if df_list:
            print("   Ghép các DataFrame...")
            df_all = pd.concat(df_list, ignore_index=True)
            print(f"✅ Đọc và ghép {len(df_list)} file thành công.")
            print(f"👉 Tổng số dòng: {df_all.shape[0]:,}")
            print(f"⏱️ Thời gian đọc và ghép: {time.time() - start_read_time:.2f} giây")
            # Hiển thị thông tin bộ nhớ (tùy chọn)
            # df_all.info(memory_usage='deep')
        else:
            print("❌ Không đọc được file nào thành công.")
            df_all = pd.DataFrame()

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
    df_all = pd.DataFrame()
except Exception as e:
    print(f"❌ Lỗi không xác định khi đọc file/thư mục: {e}")
    df_all = pd.DataFrame()

🔄 Bắt đầu đọc dữ liệu Parquet...
🔍 Tìm thấy 4 file Parquet. Bắt đầu đọc...
   Ghép các DataFrame...
✅ Đọc và ghép 4 file thành công.
👉 Tổng số dòng: 18,404,864
⏱️ Thời gian đọc và ghép: 6.71 giây


In [2]:
# Đổi tên cột ENDTIME thành TIME để dễ hiểu hơn
df_all = df_all.rename(columns={"ENDTIME": "TIME"})

# Chuyển đổi kiểu dữ liệu cho cột TIME nếu cột này tồn tại
if "TIME" in df_all.columns:
    df_all["TIME"] = pd.to_datetime(df_all["TIME"], errors="coerce", format="%Y-%m-%d %H:%M:%S")
    
    # Số lượng dòng ban đầu trước khi lọc
    initial_rows = len(df_all)
    print(f"🔍 Tổng số dòng trước khi lọc theo độ phân giải 30 phút: {initial_rows:,}")
    
    # Lọc các dòng dữ liệu có phút là 0 hoặc 30 và giây là 0
    valid_time_mask = (
        ((df_all["TIME"].dt.minute == 0) | (df_all["TIME"].dt.minute == 30)) 
        & (df_all["TIME"].dt.second == 0)
    )
    
    # Lưu những dòng không hợp lệ vào df_removed trước khi lọc
    df_removed = df_all[~valid_time_mask].copy()
    
    # Áp dụng bộ lọc vào df_all
    df_all = df_all[valid_time_mask]
    
    # Sắp xếp dữ liệu theo CTDL, NMTD và TIME
    df_all = df_all.sort_values(by=["CTDL", "NMTD", "TIME"])
    
    # Báo cáo kết quả
    removed_rows = initial_rows - len(df_all)
    print(f"⚠️ Đã loại bỏ {removed_rows:,} dòng có thời gian không theo độ phân giải 30 phút chẵn")
    print(f"✅ Còn lại {len(df_all):,} dòng dữ liệu hợp lệ")
    print(f"📊 Tỷ lệ dữ liệu hợp lệ: {(len(df_all) / initial_rows * 100):.2f}%")
    
    if not df_removed.empty:
        print(f"💾 Đã lưu {len(df_removed):,} dòng không hợp lệ vào df_removed")
        # Hiển thị phân phối các giá trị phút không hợp lệ
        invalid_minutes = df_removed["TIME"].dt.minute.value_counts().sort_index()
        if not invalid_minutes.empty:
            print("\nPhân phối giá trị phút không hợp lệ:")
            print(invalid_minutes)
            
    # Gán kết quả cho df để sử dụng trong các cell tiếp theo
    df = df_all.copy()

🔍 Tổng số dòng trước khi lọc theo độ phân giải 30 phút: 18,404,864
⚠️ Đã loại bỏ 60,706 dòng có thời gian không theo độ phân giải 30 phút chẵn
✅ Còn lại 18,344,158 dòng dữ liệu hợp lệ
📊 Tỷ lệ dữ liệu hợp lệ: 99.67%
💾 Đã lưu 60,706 dòng không hợp lệ vào df_removed

Phân phối giá trị phút không hợp lệ:
TIME
0       284
1       270
2       285
3       263
4       387
5       367
6       362
7       296
8       226
9       241
10      196
11      194
12      215
13      209
14      212
15    22933
16      207
17      174
18      186
19      199
20      205
21      204
22      184
23      207
24      230
25      249
26      299
27      382
28      373
29      359
30      386
31      387
32      305
33      294
34      371
35      381
36      440
37      275
38      239
39      221
40      226
41      198
42      203
43      207
44      147
45    22900
46      215
47      175
48      195
49      180
50      185
51      183
52      218
53      201
54      229
55      236
56      242
57      3

In [3]:
df_removed

,CTDL,NMTD,MADIEMDO,TIME,CS
4265821,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ PÁC CÁP,G2A234S000M331,2021-07-27 08:05:00,0.0
4265823,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ PÁC CÁP,G2A234S000M331,2021-07-27 08:15:00,0.0
4265828,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ PÁC CÁP,G2A234S000M331,2021-07-27 08:40:00,0.0
4265894,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ PÁC CÁP,G2A234S000M331,2021-08-28 16:35:00,0.0
4265895,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ PÁC CÁP,G2A234S000M331,2021-08-28 16:40:00,0.0
...,...,...,...,...,...
18393917,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M131,2024-04-14 17:52:32,0.0
18403846,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M132,2024-07-27 20:09:51,28.0
18403847,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M132,2024-07-27 20:42:37,0.0
18403853,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M131,2024-07-27 20:09:33,37.0


In [4]:
df_ref = pd.read_excel(r"C:\Khue\TDN\data\processed\DanhsachNM_sanluong_20250423.xlsx", sheet_name="Sheet1", usecols="A:D")
df_ref

,CTDL,NMTD,MADIEMDO,Lỗi đơn vị
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,0.0
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ PÁC CÁP,G2A234S000M331,0.0
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000M371,0.0
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ THƯỢNG ÂN,G2A122S000M371,0.0
4,CTY ĐIỆN LỰC BẮC KẠN,THÁC GIỀNG 1,G2A219S000M371,0.0
...,...,...,...,...
352,CTY ĐIỆN LỰC YÊN BÁI,NMTĐ Thác Cá 1,G2A247S000M131,1.0
353,CTY ĐIỆN LỰC YÊN BÁI,NMTĐ Thác Cá 2,G2A222S000M131,1.0
354,CTY ĐIỆN LỰC YÊN BÁI,NMTĐ THÀO SA CHẢI,G2A286S000M371,0.0
355,CTY ĐIỆN LỰC YÊN BÁI,NMTĐ TRẠM TẤU,G2A154S000M175,1.0


In [5]:
# Tôi muốn merge cột "Lỗi đơn vị" từ df_ref vào df sử dụng cột chung "MADIEMDO"
# Đảm bảo cột "MADIEMDO" trong df_ref là kiểu string
df_ref["MADIEMDO"] = df_ref["MADIEMDO"].astype(str)
# Đảm bảo cột "MADIEMDO" trong df là kiểu string
df["MADIEMDO"] = df["MADIEMDO"].astype(str)
# Merge df_ref vào df dựa trên cột "MADIEMDO"
df_final = df.merge(df_ref[["MADIEMDO", "Lỗi đơn vị"]], on="MADIEMDO", how="left")
df_final 

,CTDL,NMTD,MADIEMDO,TIME,CS,Lỗi đơn vị
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 00:30:00,951.300,0.0
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 01:00:00,1027.300,0.0
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 01:30:00,151.640,0.0
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 02:00:00,0.000,0.0
4,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 02:30:00,0.000,0.0
...,...,...,...,...,...,...
18344153,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 08:00:00,0.066,3.0
18344154,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 08:30:00,0.066,3.0
18344155,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 09:00:00,0.066,3.0
18344156,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 09:30:00,0.066,3.0


In [7]:
# Xóa các hàng có giá trị null ở cột 'Lỗi đơn vị'
rows_before = len(df_final)
df_final = df_final[df_final['Lỗi đơn vị'].notnull()]
rows_after = len(df_final)
print(f"Đã xóa {rows_before - rows_after:,} dòng có giá trị null ở cột 'Lỗi đơn vị'")

Đã xóa 638,056 dòng có giá trị null ở cột 'Lỗi đơn vị'


In [8]:
# Chuẩn hóa cột 'CS' dựa trên giá trị 'Lỗi đơn vị'
def normalize_cs(row):
    error = row['Lỗi đơn vị']
    cs = row['CS']
    if error == 0:
        return cs
    elif error == 1:
        return cs * 100
    elif error == 2:
        return cs * 1e4
    elif error == 3:
        return cs * 1e5
    elif error == 4:
        return cs * 1e6
    elif error == 5:
        return cs / 10
    else:
        return cs  # Giữ nguyên nếu giá trị khác

df_final['CS_chuanhoa'] = df_final.apply(normalize_cs, axis=1)

In [9]:
df_final

,CTDL,NMTD,MADIEMDO,TIME,CS,Lỗi đơn vị,CS_chuanhoa
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 00:30:00,951.300,0.0,951.30
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 01:00:00,1027.300,0.0,1027.30
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 01:30:00,151.640,0.0,151.64
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 02:00:00,0.000,0.0,0.00
4,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371,2021-01-01 02:30:00,0.000,0.0,0.00
...,...,...,...,...,...,...,...
18344153,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 08:00:00,0.066,3.0,6600.00
18344154,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 08:30:00,0.066,3.0,6600.00
18344155,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 09:00:00,0.066,3.0,6600.00
18344156,CTY ĐIỆN LỰC ĐIỆN BIÊN,NMTĐ Đề Bâu,G2A283S000M331,2024-08-08 09:30:00,0.066,3.0,6600.00


In [12]:
# export df_final to parquet
output_folder = r"C:\Khue\TDN\data\interim"
output_file = os.path.join(output_folder, "sanluong_chuanhoa.parquet")
# export df_final to csv
output_file = os.path.join(output_folder, "sanluong_chuanhoa.csv")
df_final.to_parquet(output_file, index=False, engine="pyarrow")
df_final.to_csv(output_file, index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

def save_df_to_excel_multiple_sheets(df, file_name='du_lieu_solar.xlsx', max_rows_per_sheet=1048576):
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    total_rows = len(df)
    num_sheets = (total_rows // max_rows_per_sheet) + (1 if total_rows % max_rows_per_sheet else 0)

    for i in range(num_sheets):
        start_row = i * max_rows_per_sheet
        end_row = min(start_row + max_rows_per_sheet, total_rows)
        sheet_df = df.iloc[start_row:end_row]
        sheet_name = f'Sheet_{i+1}'
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f'Đã ghi {sheet_name} ({start_row} → {end_row})')

    writer.close()
    print(f'✅ Đã lưu toàn bộ dữ liệu vào file: {file_name}')
    
save_df_to_excel_multiple_sheets(df_final, 'sanluong_chuanhoa.xlsx')

Đã ghi Sheet_1 (0 → 1048576)
Đã ghi Sheet_2 (1048576 → 2097152)
Đã ghi Sheet_3 (2097152 → 3145728)
Đã ghi Sheet_4 (3145728 → 4194304)
Đã ghi Sheet_5 (4194304 → 5242880)
Đã ghi Sheet_6 (5242880 → 6291456)
Đã ghi Sheet_7 (6291456 → 7340032)
